In [ ]:
# some random data
import numpy as np
size = 100
np.random.seed(0)
x_data = np.arange(size)
y_data = np.cumsum(np.random.randn(size)  * 100.0)

In [ ]:
# imports
import ipywidgets as widgets
from bqplot import LinearScale, Scatter, Figure, Axis, Lines
from bqplot.interacts import MouseInteraction

In [ ]:
x_scale = LinearScale()
y_scale = LinearScale()
scatter_chart = Scatter(x=x_data, y=y_data, scales={'x': x_scale, 'y': y_scale})
x_axis = Axis(label='X', scale=x_scale)
y_axis = Axis(label='Y', scale=y_scale, orientation='vertical')
patch = Lines(x=[[20, 30, 40, 30]],  y=[[0, 200, 600]],
              fill_colors=['orange'], fill='inside',
              close_path=True,
              scales={'x': x_scale, 'y': y_scale},
              )

In [ ]:
output = widgets.Output()

In [ ]:
mouse_interaction = MouseInteraction(x_scale=x_scale, y_scale=y_scale, cursor='crosshair')
fig = Figure(marks=[scatter_chart, patch], title='Drag to draw', axes=[x_axis, y_axis], layout={'width': '500px'},
             interaction=mouse_interaction)
fig

In [ ]:
xlist = []
ylist = []
last_x = 0
last_y = 0
move_mode = False

def mouse_msg(self, msg, buffers):
    global xlist, ylist, move_mode, last_x, last_y
    with output:
        name = msg['event']
        domain = msg['domain']
        x, y = domain['x'], domain['y']
        if name == 'click':
            print(name, x, y)
            scatter_chart.x = scatter_chart.x.tolist() + [x]
            scatter_chart.y = scatter_chart.y.tolist() + [y]            
            scatter_chart.colors = ['red'] if scatter_chart.colors == ['steelblue'] else ['steelblue']
        if name == 'dblclick':
            move_mode = not move_mode
            fig.title = 'Drag to move' if move_mode else 'Drag to draw'
            mouse_interaction.cursor = 'grab' if move_mode else 'crosshair'
        if move_mode:
            if name == 'dragstart':
                last_x, last_y = x, y
                patch.fill_colors = ['blue']
            elif name == 'dragmove':
                dx = x - last_x
                dy = y - last_y
                last_x, last_y = x, y
                patch.x = patch.x + dx
                patch.y = patch.y + dy
                patch.fill_colors = ['pink']
            elif name == 'dragend':
                patch.fill_colors = ['purple']
        else:
            if name == 'dragstart':
                xlist = [x]
                ylist = [y]
                patch.fill_colors = ['orange']
            elif name == 'dragmove':
                xlist.append(x)
                ylist.append(y)
                patch.x = xlist
                patch.y = ylist
            else:
                patch.fill_colors = ['green']
mouse_interaction.on_msg(mouse_msg)

In [ ]:
output